In [13]:
import pandas as pd
import nasdaqdatalink as nd
import sqlite3
import os
import gzip
import shutil
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)


from dotenv import load_dotenv
load_dotenv()

True

In [15]:
NASDAQ_API = os.environ.get('NASDAQ_API')
nd.ApiConfig.api_key = NASDAQ_API

In [14]:
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_DB = os.getenv("MYSQL_DB")

DATABASE_URL = f"mysql+mysqlconnector://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}/{MYSQL_DB}"


In [21]:
def download_data(table, **filters):
    data = nd.get_table(table, **filters)
    return data

def save_to_csv(dataframe, file_path):
    dataframe.to_csv(file_path, index=False)

def compress_file(file_path):
    with open(file_path, 'rb') as f_in:
        with gzip.open(file_path + '.gz', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    os.remove(file_path)

def save_to_db(dataframe, table_name):
    engine = create_engine(DATABASE_URL)
    try:
        dataframe.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"Data saved to {table_name} table successfully.")
    except SQLAlchemyError as e:
        print(f"Error: {str(e)}")

def retrieve_data(query):
    engine = create_engine(DATABASE_URL)
    with engine.connect() as connection:
        result = pd.read_sql(query, connection)
    return result

In [16]:
table = 'QDL/FON'
filters = {
    'date': '2019-10-08', 
    'contract_code': '967654'
}
data = download_data(table, **filters)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.nasdaq.com:443
DEBUG:urllib3.connectionpool:https://data.nasdaq.com:443 "GET /api/v3/datatables/QDL/FON?date=2019-10-08&contract_code=967654 HTTP/1.1" 200 None


In [18]:
data.head()

,contract_code,type,date,market_participation,producer_merchant_processor_user_longs,producer_merchant_processor_user_shorts,swap_dealer_longs,swap_dealer_shorts,swap_dealer_spreads,money_manager_longs,money_manager_shorts,money_manager_spreads,other_reportable_longs,other_reportable_shorts,other_reportable_spreads,total_reportable_longs,total_reportable_shorts,non_reportable_longs,non_reportable_shorts
None,,,,,,,,,,,,,,,,,,,
0,967654,F_OTR_OI,2019-10-08,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,967654,F_OTR_NT,2019-10-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,967654,F_OTR,2019-10-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,967654,FO_OTR_OI,2019-10-08,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,967654,FO_OTR_NT,2019-10-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [25]:
save_to_db(data, 'QDL_FON')

Data saved to QDL_FON table successfully.


C:\Users\yoyo\AppData\Local\Temp\ipykernel_34624\2460718170.py:17: UserWarning: The provided table name 'QDL_FON' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  dataframe.to_sql(table_name, engine, if_exists='replace', index=False)


In [26]:
query = "SELECT * FROM QDL_FON WHERE date='2019-10-08'"
retrieved_data = retrieve_data(query)
retrieved_data.head()

,contract_code,type,date,market_participation,producer_merchant_processor_user_longs,producer_merchant_processor_user_shorts,swap_dealer_longs,swap_dealer_shorts,swap_dealer_spreads,money_manager_longs,money_manager_shorts,money_manager_spreads,other_reportable_longs,other_reportable_shorts,other_reportable_spreads,total_reportable_longs,total_reportable_shorts,non_reportable_longs,non_reportable_shorts
0,967654,F_OTR_OI,2019-10-08,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,967654,F_OTR_NT,2019-10-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,967654,F_OTR,2019-10-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,967654,FO_OTR_OI,2019-10-08,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,967654,FO_OTR_NT,2019-10-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [ ]:
# TEST

In [27]:
def test_download_data():
    table = 'QDL/FON'
    filters = {'date': '2019-10-08', 'contract_code': '967654'}
    data = download_data(table, **filters)
    assert not data.empty, "Downloaded data is empty"

test_download_data()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.nasdaq.com:443
DEBUG:urllib3.connectionpool:https://data.nasdaq.com:443 "GET /api/v3/datatables/QDL/FON?date=2019-10-08&contract_code=967654 HTTP/1.1" 200 None


In [28]:
def test_save_to_db():
    table = 'QDL/FON'
    filters = {'date': '2019-10-08', 'contract_code': '967654'}
    data = download_data(table, **filters)
    save_to_db(data, 'test_table')
    # Further assertions to verify the data saved to the database

test_save_to_db()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.nasdaq.com:443
DEBUG:urllib3.connectionpool:https://data.nasdaq.com:443 "GET /api/v3/datatables/QDL/FON?date=2019-10-08&contract_code=967654 HTTP/1.1" 200 None


Data saved to test_table table successfully.


In [29]:
def test_retrieve_data():
    query = "SELECT * FROM test_table WHERE date='2019-10-08'"
    data = retrieve_data(query)
    assert not data.empty, "Retrieved data is empty"

test_retrieve_data()

In [27]:
# nd.Database('WIKI').bulk_download_to_file('/data')


In [26]:
db = nd.Database('WIKI')
db.database_code

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.nasdaq.com:443
DEBUG:urllib3.connectionpool:https://data.nasdaq.com:443 "GET /api/v3/databases/WIKI HTTP/1.1" 200 None


'WIKI'

In [21]:
dt = nd.Datatable('ZACKS/FC')

In [24]:
dt.data_fields()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.nasdaq.com:443
DEBUG:urllib3.connectionpool:https://data.nasdaq.com:443 "GET /api/v3/datatables/ZACKS/FC/metadata HTTP/1.1" 200 None


['vendor_code',
 'datatable_code',
 'name',
 'description',
 'columns',
 'filters',
 'primary_key',
 'premium',
 'status',
 'data_version']

In [30]:
database = nd.Database('WIKI')
database.data_fields()


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.nasdaq.com:443
DEBUG:urllib3.connectionpool:https://data.nasdaq.com:443 "GET /api/v3/databases/WIKI HTTP/1.1" 200 None


['id',
 'name',
 'database_code',
 'description',
 'datasets_count',
 'downloads',
 'premium',
 'image',
 'favorite',
 'url_name',
 'exclusive']

In [31]:
databases = nd.Datatable('RATE')
databases.values

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.nasdaq.com:443
DEBUG:urllib3.connectionpool:https://data.nasdaq.com:443 "GET /api/v3/databases HTTP/1.1" 410 None


DataLinkError: (Status 410) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@data.nasdaq.com.